In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


In [13]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id="d76a78cc7dee4eb7927f0e2b7871e2b5",
        client_secret="027c85dee23340ecbcca23cb0c7a0ac8"
    )
)


In [14]:
import pandas as pd

df = pd.read_csv("../data/processed/spotify_featured.csv")
df.head()


,ts,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,spotify_track_uri,episode_name,...,day_name,month,content_type,play_minutes,skip_flag,prev_ts,time_gap,new_session,session_id,late_night
0,2021-04-06 14:15:57+00:00,"Android OS 11 API 30 (samsung, SM-M215F)",8820,IN,49.206.35.80,00:00 (Zero O’Clock),BTS,MAP OF THE SOUL : 7,spotify:track:4AlihYDqxXshKhvh5tnMfP,NaN,...,Tuesday,2021-04,Song,0.147000,True,NaN,NaN,False,0,False
1,2021-04-06 14:17:44+00:00,"Android OS 11 API 30 (samsung, SM-M215F)",107594,IN,49.206.35.80,Interlude : Shadow,BTS,MAP OF THE SOUL : 7,spotify:track:2Nib4r0saYQf2pUV4RFDhl,NaN,...,Tuesday,2021-04,Song,1.793233,False,2021-04-06 14:15:57+00:00,1.783333,False,0,False
2,2021-04-06 14:17:51+00:00,"Android OS 11 API 30 (samsung, SM-M215F)",6237,IN,49.206.35.80,One Day At A Time,ATEEZ,ZERO : FEVER Part.1,spotify:track:1eFk8R4OobU7kaAZdfYBKz,NaN,...,Tuesday,2021-04,Song,0.103950,True,2021-04-06 14:17:44+00:00,0.116667,False,0,False
3,2021-04-06 14:18:09+00:00,"Android OS 11 API 30 (samsung, SM-M215F)",18266,IN,49.206.35.80,Precious (Overture),ATEEZ,TREASURE EP.FIN : All To Action,spotify:track:6gbhjkoubkuIzKSEV37suF,NaN,...,Tuesday,2021-04,Song,0.304433,True,2021-04-06 14:17:51+00:00,0.300000,False,0,False
4,2021-04-06 14:21:27+00:00,"Android OS 11 API 30 (samsung, SM-M215F)",197390,IN,49.206.35.80,ILLUSION,ATEEZ,TREASURE EP.3 : One To All,spotify:track:4hQHA562yLkVKlakkOkz5l,NaN,...,Tuesday,2021-04,Song,3.289833,False,2021-04-06 14:18:09+00:00,3.300000,False,0,False


In [15]:
def get_album_cover(uri):
    try:
        track = sp.track(uri)
        images = track['album']['images']
        if images:
            return images[0]['url']  # highest resolution
        else:
            return None
    except:
        return None


In [16]:
df_test = df.head(50).copy()
df_test['album_cover_url'] = df_test['spotify_track_uri'].apply(get_album_cover)

df_test[['track_name','album_cover_url']].head()


,track_name,album_cover_url
0,00:00 (Zero O’Clock),None
1,Interlude : Shadow,None
2,One Day At A Time,None
3,Precious (Overture),None
4,ILLUSION,None


In [17]:
df_test[['track_name', 'spotify_track_uri']].head(10)


,track_name,spotify_track_uri
0,00:00 (Zero O’Clock),spotify:track:4AlihYDqxXshKhvh5tnMfP
1,Interlude : Shadow,spotify:track:2Nib4r0saYQf2pUV4RFDhl
2,One Day At A Time,spotify:track:1eFk8R4OobU7kaAZdfYBKz
3,Precious (Overture),spotify:track:6gbhjkoubkuIzKSEV37suF
4,ILLUSION,spotify:track:4hQHA562yLkVKlakkOkz5l
5,Pacemaker,spotify:track:53StevmjK3N3aYHwoxRsBQ
6,Control,spotify:track:3SKzTIl9mW0DadTys9SFo3
7,Delicate,spotify:track:6NFyWDv5CjfwuzoCkw47Xf
8,So What,spotify:track:4CvId64YZSB0RfRdfzPIQU
9,FAKE LOVE,spotify:track:5Sn2fm8aisVlR7S6u1KW57


In [18]:
def get_album_cover(row):
    try:
        # 1. Try using track URI first
        if pd.notnull(row['spotify_track_uri']):
            track = sp.track(row['spotify_track_uri'])
            images = track['album']['images']
            if images:
                return images[0]['url']

        # 2. Fallback: search by track + artist
        query = f"track:{row['track_name']} artist:{row['artist_name']}"
        result = sp.search(q=query, type='track', limit=1)

        if result['tracks']['items']:
            images = result['tracks']['items'][0]['album']['images']
            if images:
                return images[0]['url']

        return None

    except:
        return None


In [19]:
df_test = df.head(20).copy()
df_test['album_cover_url'] = df_test.apply(get_album_cover, axis=1)

df_test[['track_name', 'album_cover_url']]


,track_name,album_cover_url
0,00:00 (Zero O’Clock),None
1,Interlude : Shadow,None
2,One Day At A Time,None
3,Precious (Overture),None
4,ILLUSION,None
5,Pacemaker,None
6,Control,None
7,Delicate,None
8,So What,None
9,FAKE LOVE,None


In [20]:
df[['spotify_track_uri']].head(5)


,spotify_track_uri
0,spotify:track:4AlihYDqxXshKhvh5tnMfP
1,spotify:track:2Nib4r0saYQf2pUV4RFDhl
2,spotify:track:1eFk8R4OobU7kaAZdfYBKz
3,spotify:track:6gbhjkoubkuIzKSEV37suF
4,spotify:track:4hQHA562yLkVKlakkOkz5l


In [21]:
df['track_id'] = df['spotify_track_uri'].str.split(':').str[-1]


In [22]:
df[['spotify_track_uri', 'track_id']].head()


,spotify_track_uri,track_id
0,spotify:track:4AlihYDqxXshKhvh5tnMfP,4AlihYDqxXshKhvh5tnMfP
1,spotify:track:2Nib4r0saYQf2pUV4RFDhl,2Nib4r0saYQf2pUV4RFDhl
2,spotify:track:1eFk8R4OobU7kaAZdfYBKz,1eFk8R4OobU7kaAZdfYBKz
3,spotify:track:6gbhjkoubkuIzKSEV37suF,6gbhjkoubkuIzKSEV37suF
4,spotify:track:4hQHA562yLkVKlakkOkz5l,4hQHA562yLkVKlakkOkz5l


In [23]:
track_id = df.loc[0, 'track_id']
sp.track(track_id)


SpotifyOauthError: error: invalid_client, error_description: Invalid client

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(
    auth_manager=SpotifyClientCredentials(
        client_id="d76a78cc7dee4eb7927f0e2b7871e2b5",
        client_secret="027c85dee23340ecbcca23cb0c7a0ac8"
    )
)

sp.search(q="Arijit Singh", type="artist", limit=1)
